dependencies

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
#import neattext.functions as nfx
from sklearn.naive_bayes import MultinomialNB
# Transformers
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

c:\Users\manya\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [13]:
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [11]:
from transformers import AutoTokenizer

# Load Distilbert Tokenizer 
#model_ckpt = "distilbert-base-uncased"
model_ckpt = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [12]:
# Encode Our Example Text
encoded_text = tokenizer("The movie was not good")
tokens = tokenizer.convert_ids_to_tokens(encoded_text.input_ids)
print(encoded_text,len(encoded_text.input_ids))
print(tokens)

{'input_ids': [101, 1996, 3185, 2001, 2025, 2204, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]} 7
['[CLS]', 'the', 'movie', 'was', 'not', 'good', '[SEP]']


Cleaning Data

In [24]:
#data = pd.read_csv(r'.\Eminem_Lyrics.csv', sep='\t', comment='#', encoding="ISO-8859-1")
data = pd.read_csv(r'.\spotify_millsongdata.csv')

In [25]:
data.head(20)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...
5,ABBA,Burning My Bridges,/a/abba/burning+my+bridges_20003011.html,"Well, you hoot and you holler and you make me ..."
6,ABBA,Cassandra,/a/abba/cassandra_20002811.html,Down in the street they're all singing and sho...
7,ABBA,Chiquitita,/a/abba/chiquitita_20002978.html,"Chiquitita, tell me what's wrong \r\nYou're e..."
8,ABBA,Crazy World,/a/abba/crazy+world_20003013.html,I was out with the morning sun \r\nCouldn't s...
9,ABBA,Crying Over You,/a/abba/crying+over+you_20177611.html,I'm waitin' for you baby \r\nI'm sitting all ...


In [26]:
def clean_text(text, max_length=578):
    cleaned_text = ' '.join(text.split())
    cleaned_text = cleaned_text.lower()
    cleaned_text = cleaned_text.translate(str.maketrans("", "", string.punctuation))
    cleaned_text = ''.join([i for i in cleaned_text if not i.isdigit()])
    cleaned_text = ''.join([i for i in cleaned_text if i.isalpha() or i.isspace()])
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(cleaned_text)
    cleaned_text = ' '.join([word for word in word_tokens if word.lower() not in stop_words])
    
    padded_text = cleaned_text.ljust(max_length)
    
    return padded_text

In [27]:
data['text'] = data['text'].apply(clean_text)

In [28]:
data.shape

(57650, 4)

In [29]:
data.head(20)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,look face wonderful face means something speci...
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,take easy please touch gently like summer even...
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,ill never know go put lousy rotten show boy to...
3,ABBA,Bang,/a/abba/bang_20598415.html,making somebody happy question give take learn...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,making somebody happy question give take learn...
5,ABBA,Burning My Bridges,/a/abba/burning+my+bridges_20003011.html,well hoot holler make mad ive always heel holy...
6,ABBA,Cassandra,/a/abba/cassandra_20002811.html,street theyre singing shouting staying alive t...
7,ABBA,Chiquitita,/a/abba/chiquitita_20002978.html,chiquitita tell whats wrong youre enchained so...
8,ABBA,Crazy World,/a/abba/crazy+world_20003013.html,morning sun couldnt sleep thought id take walk...
9,ABBA,Crying Over You,/a/abba/crying+over+you_20177611.html,im waitin baby im sitting alone feel cold with...


In [30]:
#export clean data as csv
data.to_csv('SpotifyLyricsCleaned.csv', sep="\t", index=False)

Train Test Splits

In [18]:
# Step 2: Splitting the Data
train_texts, test_texts, train_labels, test_labels = train_test_split(
    data['Clean_Text'], data['Emotion'], test_size=0.2, random_state=42
)

In [28]:
train_texts

27845                       Yes  anniversary come  Cheers 
25699                                        think bloody 
24688    Peace state fear kind unknown John Buchan Fill...
32367    fat bitches complain weight TOLD EAT MCDONALDS...
13168    Heb een lolly gekregen met 12 lollys erin en 1...
                               ...                        
16850                 Damn Im going stuck american time Xx
6265                                haha ill tie guys post
11284    jump wagon Worse case scenario die Till #alive...
860                                           bby plate :(
15795                                  friends try hurt me
Name: Clean_Text, Length: 27833, dtype: object

In [25]:
encoded_text = tokenizer(train_texts[0][0])
tokens = tokenizer.convert_ids_to_tokens(encoded_text.input_ids)
print(encoded_text,len(encoded_text.input_ids))
print(tokens)

TypeError: 'float' object is not subscriptable

In [36]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split

In [56]:
# Load the dataset
df = pd.read_csv(r'.\emotion_dataset_2.csv')
df['Clean_Text'] = df['Clean_Text'].fillna('')
df = df.head(800)

# Tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Encode text data
encoded_data = tokenizer(df['Clean_Text'].tolist(), padding=True, truncation=True, return_tensors='tf')

In [57]:
encoded_data

{'input_ids': <tf.Tensor: shape=(800, 46), dtype=int32, numpy=
array([[  101,   102,     0, ...,     0,     0,     0],
       [  101, 10878,  2552, ...,     0,     0,     0],
       [  101,  2126,  2188, ...,     0,     0,     0],
       ...,
       [  101,  8837,  2627, ...,     0,     0,     0],
       [  101,  8741, 20934, ...,     0,     0,     0],
       [  101,  9826,  2360, ...,     0,     0,     0]])>, 'token_type_ids': <tf.Tensor: shape=(800, 46), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])>, 'attention_mask': <tf.Tensor: shape=(800, 46), dtype=int32, numpy=
array([[1, 1, 0, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>}

In [58]:
# Convert TensorFlow tensors to numpy arrays
X_data = encoded_data['input_ids'].numpy()
y_data = df['Emotion'].values

In [59]:
y_data

array(['neutral', 'joy', 'sadness', 'joy', 'joy', 'fear', 'sadness',
       'surprise', 'surprise', 'surprise', 'anger', 'sadness', 'joy',
       'fear', 'anger', 'sadness', 'surprise', 'joy', 'joy', 'surprise',
       'sadness', 'joy', 'fear', 'anger', 'fear', 'sadness', 'surprise',
       'joy', 'sadness', 'joy', 'joy', 'fear', 'fear', 'shame', 'fear',
       'joy', 'sadness', 'surprise', 'sadness', 'surprise', 'surprise',
       'anger', 'fear', 'sadness', 'anger', 'fear', 'sadness', 'joy',
       'anger', 'joy', 'joy', 'fear', 'sadness', 'sadness', 'sadness',
       'sadness', 'fear', 'fear', 'sadness', 'sadness', 'surprise',
       'neutral', 'joy', 'joy', 'surprise', 'fear', 'fear', 'fear',
       'neutral', 'fear', 'joy', 'joy', 'fear', 'joy', 'sadness', 'fear',
       'neutral', 'anger', 'fear', 'fear', 'fear', 'joy', 'anger',
       'anger', 'joy', 'surprise', 'surprise', 'surprise', 'anger',
       'surprise', 'disgust', 'anger', 'joy', 'fear', 'surprise',
       'sadness', '

In [60]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

In [61]:
# Convert labels to one-hot encoding
#num_classes = len(df['Emotion'].unique())
#y_train_one_hot = tf.keras.utils.to_categorical(y_train, num_classes)
#y_test_one_hot = tf.keras.utils.to_categorical(y_test, num_classes)

from sklearn.preprocessing import LabelEncoder

# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [62]:
# Load pre-trained BERT model
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_classes)

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = ['accuracy']
bert_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [63]:
# Train the model
history = bert_model.fit(X_train, y_train_encoded, validation_data=(X_test, y_test_encoded), epochs=3, batch_size=32)

Epoch 1/3
20/20 [==============================] - 377s 17s/step - loss: 1.8802 - accuracy: 0.2625 - val_loss: 1.8326 - val_accuracy: 0.2688
Epoch 2/3
20/20 [==============================] - 339s 17s/step - loss: 1.7883 - accuracy: 0.2828 - val_loss: 1.8129 - val_accuracy: 0.2750
Epoch 3/3
20/20 [==============================] - 339s 17s/step - loss: 1.7501 - accuracy: 0.3000 - val_loss: 1.7899 - val_accuracy: 0.2625


In [65]:
# Evaluate the model
loss, accuracy = bert_model.evaluate(X_test, y_test_encoded)
print(f'Loss: {loss}, Accuracy: {accuracy}')

# Save the model
bert_model.save_pretrained('bert_classification_model')

5/5 [==============================] - 21s 4s/step - loss: 1.7899 - accuracy: 0.2625
Loss: 1.7899296283721924, Accuracy: 0.26249998807907104


Annotating Datasets Using an Exisitng Bert Model

In [31]:
from transformers import RobertaTokenizerFast, TFRobertaForSequenceClassification, pipeline
import pandas as pd

# Load the model
tokenizer = RobertaTokenizerFast.from_pretrained("arpanghoshal/EmoRoBERTa")
model = TFRobertaForSequenceClassification.from_pretrained("arpanghoshal/EmoRoBERTa")

# Load the dataset
# df = pd.read_csv("Eminem_Lyrics.csv", sep='\t', comment='#', encoding="ISO-8859-1")
df = pd.read_csv(r'.\EminemLyricsCleaned.csv', sep='\t', comment='#', encoding="ISO-8859-1")

# Now we need to go through each song's lyrics one by one and use the model to annotate them with the corresponding emotion
emotion = pipeline('sentiment-analysis', 
                    model='arpanghoshal/EmoRoBERTa')

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at arpanghoshal/EmoRoBERTa.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at arpanghoshal/EmoRoBERTa.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [32]:
# Drop unnecessary columns - link
df = df.drop(columns=['Album_URL'])
df = df.drop(columns=['Views'])
df = df.drop(columns=['Release_date'])

In [34]:
# truncate the lyrics to 512 tokens so that when we call the model it doesn't run out of memory
def truncate(text):
    return text[:512]

# Annotate the lyrics with the corresponding emotion
df['Lyrics'] = df['Lyrics'].apply(truncate)

# We will save the annotated lyrics in a new column in the dataframe
df['Emotion'] = df['Lyrics'].apply(emotion)
df['Emotion'] = df['Emotion'].apply(lambda x: x[0]['label'])

# Save the annotated dataframe to a new CSV file
df.to_csv('EminemLyricsAnnotated.csv', sep='\t', index=False)

In [36]:
df.head(30)

,Album_Name,Song_Name,Lyrics,Unnamed: 6,Emotion
0,Music To Be Murdered By: Side B,Alfred (Intro),intro alfred hitchcock thus far album provided...,NaN,neutral
1,Music To Be Murdered By: Side B,Black Magic,chorus skylar grey eminem black magic night wa...,NaN,neutral
2,Music To Be Murdered By: Side B,AlfredÂs Theme,verse check mic check check one two give extra...,NaN,neutral
3,Music To Be Murdered By: Side B,Tone Deaf,intro yeah im sorry huh say oh cant hear ear i...,NaN,remorse
4,Music To Be Murdered By: Side B,Book of Rhymes,intro dont smile dont frown get get nah came w...,NaN,anger
5,Music To Be Murdered By: Side B,Favorite Bitch,intro sly pyper eminem nah real know im sayin ...,NaN,neutral
6,Music To Be Murdered By: Side B,Guns Blazing,intro sly pyper dr dre time face music stay ex...,NaN,annoyance
7,Music To Be Murdered By: Side B,Gnat,intro yeah yeah sick da got dope chorus say ba...,NaN,neutral
8,Music To Be Murdered By: Side B,Higher,intro eminem yeah know ive around sure much le...,NaN,amusement
9,Music To Be Murdered By: Side B,These Demons,intro eminem yeah know loud pack haters funny ...,NaN,amusement
